In [2]:
from pathlib import Path
import pyloudnorm as pyln
import torch
import matplotlib.pyplot as plt
import numpy as np

from network import NAF
from sound_loader import sound_samples
from utilities import to_audio
import utils.irutilities as irutil

# Initialize LoudNorm object
meter = pyln.Meter(48000) # create BS.1770 meter

# Load dataset
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Loading data...')
dataset = sound_samples(num_samples=20)
posMic, posSrc, irs = irutil.loadIR(Path('mesh_rir/S32-M441_npy/'))

# Load NAF with selected configuration
net = NAF(input_dim = 416, min_xy=np.array([-1, -1]), max_xy=np.array([1, 1])).to(device)
state_dict = torch.load('saved/net_20230916_144210_loss_0.0009301.pth')
net.load_state_dict(state_dict)
net.eval()

# Generate grid for inference
num_points = 35 # for each axis
x_values = np.linspace(-1, 1, num_points)
y_values = np.linspace(-1, 1, num_points)
xx, yy = np.meshgrid(x_values, y_values)
grid_points = np.column_stack((xx.ravel(), yy.ravel()))

# Perform inference
src = torch.tensor(posSrc[0]).unsqueeze(1).to(device)
result = np.zeros(grid_points.shape[0])

print('Doing things...')
for i, point in enumerate(grid_points):
    point = np.concatenate((point, [0]))
    mic = torch.tensor(point).unsqueeze(1).to(device)
    out = net.spectrogram_at(src, mic).cpu()
    out_s = (out[:,:,0] * 40) - 40
    #out_s[out_s > -15] /= 2
    out_p = out[:,:,1] * 180

    audio = to_audio(out_s, out_p)
    loudness = np.max(np.abs(audio))
    result[i] = loudness


reshaped_array = result.reshape(35, 35)

# Create a heatmap plot
plt.imshow(reshaped_array, cmap='viridis', interpolation='nearest')
plt.colorbar()
plt.show()


Loading data...
Doing things...


In [ ]:
log_array = 20 * np.log10(reshaped_array)
plt.imshow(log_array, cmap='viridis', interpolation='nearest')
plt.colorbar()
plt.show()